<a href="https://colab.research.google.com/github/Hirunikathaluwage/RAG/blob/main/Conversational_RAG_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain_community -qU
!pip install langchain-chroma -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.7 MB/s eta 0:00:00

In [2]:
import os
from google.colab import userdata

#Initialize OpenAI APIKEY

In [3]:
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


#Use openai embedding model

In [4]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")


#Load pdf document

In [5]:
!pip install pypdf -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 4.6 MB/s eta 0:00:00


In [6]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/Final_ITP25_B6_124.pdf")

docs = loader.load()

In [7]:
len(docs)
docs[13]

Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-05-15T18:18:03+05:30', 'author': 'Geethanjali Wimalaratne', 'moddate': '2025-05-15T18:18:03+05:30', 'source': '/content/Final_ITP25_B6_124.pdf', 'total_pages': 47, 'page': 13, 'page_label': '14'}, page_content='10 \n \n1.5 Solution Overview \n \nDevelop a comprehensive web application that serves as a centralized platform, connecting patients, \npharmacies. Meditrack enables patients to effortlessly search for medicines across nearby Pharmacies, \ncheck stock availability and make reservations without the need to physically visit each location. \nPatients can add their prescription; system will extract the medici ne names through OCR and list of \nextracted medicines will be displayed after the approval of pharmacist.  \nMediTrack offers comprehensive and efficient delivery management solutions. Patients can reserve \ntheir required medicines, add th

#Split documents into chunks

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

In [9]:
len(splits)

195

In [10]:
splits[0]

Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-05-15T18:18:03+05:30', 'author': 'Geethanjali Wimalaratne', 'moddate': '2025-05-15T18:18:03+05:30', 'source': '/content/Final_ITP25_B6_124.pdf', 'total_pages': 47, 'page': 0, 'page_label': '1'}, page_content='MediTrack Centralized Pharmacy \nManagement System \n \n             Final Project Report \n \n \n \n \n \n \n \n \n \nSri Lanka Institute of Information Technology  \nIT2080 Information Technology Project \n \n \n \n             Group ITP_WD_B6_124 \n \n \n \n \n       January  2025')

#Create vector store & retriver

In [11]:
from langchain_chroma import Chroma
vectorstores = Chroma.from_documents(documents=splits, embedding=embedding_model)

In [12]:
retriver = vectorstores.as_retriever()

In [18]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always say "thanks for asking!" at the end of the answer.

{context} /* context is coming from a retriver. this takes matching chunks for user question from vector db */
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", "{question}"),
])

In [17]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Use the following pieces of context to answer the users question.\nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\nAlways say "thanks for asking!" at the end of the answer.\n\n{context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

#create the RAG chain

In [ ]:
from langchain.chains import create_retrival_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

